# Capstone Project - The Battle of the Neighborhoods

## Introduction: Business Problem

Nowadays working at office or home office is one of the most frequent setups. In most cases such type of work requires seating for multiple hours a day, which brings very unhealthy static tension to body.  
There are multiple types of sport activities to eliminate and compensiate such bad influence, one of very popular is yoga. Yoga implies combination of physical exercises with mental relaxing which also helps to decrease overall stress. Moreover most of yoga exersices do not require extreme physical pressure and are quite safe from injury prospective.  
All that said, yoga is quite nice choice for office workers who live in big urban areas.

Thus, building recomendation system for finding best suitable yoga class for office worker based on certain criteria is valuable analytical problem that perfectly fits into _Clustering_ type of Data Science problems which could be solved by unsupervised learning algorithms.

## Data

Let's import needed libraries:

In [ ]:
import pandas as pd
import folium

First we obtained geo-data about Munich boroughs from [Wikipedia](https://en.wikipedia.org/wiki/Boroughs_of_Munich) and stored it into [munich_boroughs.csv](munich_boroughs.csv) CSV-file.  
Let's visualize collected information:

In [ ]:
munich_latitude = 48.133333
munich_longitude = 11.566667
map_munich = folium.Map(location=[munich_latitude, munich_longitude], zoom_start=12)
boroughs = pd.read_csv('munich_boroughs.csv')

boroughs

Next we used Foursquare API to obtain information about Yoga classes in each boroughs:

## Methodology

TBD

## Results

TBD

## Discussion

TBD

## Conclusion

TBD